## Estrategia

Calcular en qué unidades electorales es más fácil sacar votos, a la vez que podemos asignar quién sería el beneficiario de esa ganancia. Esto permite asignar visitas y mensajes a cada candidato. Estos mensajes pueden ser de pedir cambiar el voto, y de conseguir que los no votantes voten. La facilidad de cambiar el voto es proporcional a cómo de saturada esté la votación.


## Hipótesis

1. Los resultados de las votaciones son iguales a las anteriores, o extrapolaciones de la tendencia.
2. El *efecto del esfuerzo* para ganar una % de votos satura en ambos extremos, es una gaussiana. En los extremos (cuando casi nadie o casi todos ya votan) cuesta mucho ganar votos, en el centro es más fácil.
3. Los votos ganados tienen que salir de uno de estos dos sitios: a)de votos a otros partidos, cuya resistencia a cambiar es la misma gausiana que en 2. b) de motivar a los no votantes, que votarán en primera approximación proporcionalmente a predición de resultados. I.e. la estrategia puede ser "ganar votos para un partido" o "id a votar".
4. El resultado del esfuerzo es proporcional al tamaño de la unidad de voto. Aunque el esfuerzo sea sobre el % de votos de una divisón electoral, hay divisiones con 100 candidatos, y otras con cientos de miles. 
5. Al ser España una circumscripción única para las Europeas, de listas cerradas, se puede asignar nombre a los votos ganados o perdidos en cada actuación. E.g. 10 votos más sacan al candidato X.
* Por tanto, los efectos de los esfuerzos de voto se pueden asiganr a los candidatos correspodientes en la asignación de escaños. E.g. En el minicipio X se prevé que el esfuerzo es mínimo para sacar el escaño, y el mensaje es simplemente que voten.

Por tanto, la idea es que, para cada escaño, calcular cuantos votos hacen falta para asignárselo. Luego ordenar los lugares por orden del resultado del esfuerzo. Uno podría minimizarlo con el coste del esfuerzo, pero en principio se ordena por el efecto del esfuerzo


Dado un %% de votos, se asigna un tiron  Tx=gauss(x). Este tirón se tiene que compensar con una serie de tirones de Ta,Tb,Gc, ... y N. Siendo Ta=agauss(A) lo tirones negativos hasta que se consigue Tx. Tn es el tirón de los no votantes. El tirón de los no votantes tiene el effecto proporcional a los % de votos, por lo tanto afecta a todos. 


## Datos

* Sacar los datos de elecceiones de 2014 granulares de los municipios
* Buscar los sitios de menor participación.

* Sacar las diferencias de participación.


Busque en Wikipedia, en github, en el boe, pero nadie tiene los CSV de resultados desagregados.

Al final encontré la mayoría de datos en http://elecciones.mir.es/resultados2014/ini99v.htm

Auqneu son provisionales al 99.77% me valen.
No tienen opcion a descargas todo, ni va con API que pueda ver. Pero si tienen un ejecutable con todos los datos. Al descargarlo (en wine con OSX) veo que tiene una base de datos que quizás sí tenga todos los datos desagregados. Es un .mdb (base de dato de Microsoft Office) de 132MB.

Usé esta página para exportar los datos
https://www.codeenigma.com/community/blog/using-mdbtools-nix-convert-microsoft-access-mysql

```sql
mdb-schema Europeas2014.mdb mysql > schema.sql
mkdir sql
for i in $( mdb-tables Europeas2014.mdb ); do echo $i ; mdb-export -D "%Y-%m-%d %H:%M:%S" -H -I mysql Europeas2014.mdb $i > sql/$i.sql; done
mysql -uroot -ppassword europeas < schema.sql
for i in $( ls sql/ ); do echo $i ; mysql -uroot -ppassword europeas < sql/$i ; done
```
Dará error al importar el sql Partidos, pero es un fichero raro binario, ni idea de lo que es.

Si miras la tabla "resultados" verás que pone:


In [ ]:
def init():
    %matplotlib inline
    global os,pymysql,pd
    import os
    import pymysql
    import pandas as pd
    
    global conn
    host = os.getenv('MYSQL_HOST')
    port = os.getenv('MYSQL_PORT')
    user = os.getenv('MYSQL_USER')
    password = os.getenv('MYSQL_PASSWORD')
    database = 'europeas'    
    
    conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

init()

In [ ]:
#test connection
query="select * from resultados limit 1"
df = pd.read_sql_query(query,conn)
df.to_dict('index')

Esto quiere decir que la información de los nombres está codificada. Por ejemplo para saber qué distrito es `CANDIDATURA: 0002`


In [ ]:
query="select * from candidaturas where candidatura=0002;"
df = pd.read_sql_query(query,conn)
df.to_dict('index')


## Resultados por municipio

Como los datos están consignados, si quiero saber lo que votó en mi pueblo "Soto del Barco", primero tengo que sacar su consigna:

In [ ]:
query="select * from municipios where nombre='Soto del Barco'"
df = pd.read_sql_query(query,conn)
df.to_dict('index')


In [ ]:
query="select * from municipios where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069"
df = pd.read_sql_query(query,conn)
df

Ahora puedo pedir los resultados donde se cumplen esos criterios:

In [ ]:
query="""select candidatura,votos_act 
         from resultados 
         where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069
         and votos_act>0
         order by votos_act desc;"""
df = pd.read_sql_query(query,conn)
df.to_dict('index')



y para ver esos nombres:

In [ ]:
query="""select c.sigla,votos_act,PVOTOS_ACT 
         from resultados as r 
         join candidaturas as c on r.candidatura=c.candidatura 
         where AUTONOMIA=03 and PROVINCIA=33 and MUNICIPIO=069
         and votos_act>0
         order by votos_act desc;"""
df = pd.read_sql_query(query,conn)
df.to_dict('index')


Para ver que vamos bien podemos mirar los resultados [en la web](http://elecciones.mir.es/resultados2014/99PE/DPE0333906999.htm?d=533), dentro del app:

![Soto del Barco](sotodelbarco.png)



# Histograma de % de votos por partido y Municipio

El modelo se basa en comparar el % de votos en cada unidad, con el conseguido en todas las unidades.

In [ ]:
#Algunas funciones para sacar datos del DB o hacer gráficas
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import os

def get_todos_votos():
    query="""select c.candidatura,c.sigla,r.PVOTOS_ACT,r.VOTOS_ACT,
         r.AUTONOMIA,r.PROVINCIA,r.MUNICIPIO,r.DISTRITO
         from resultados as r 
         join candidaturas as c on r.candidatura=c.candidatura 
         and r.PVOTOS_ACT >0
         order by c.sigla asc;"""
    return pd.read_sql_query(query,conn)

def get_no_votantes():
    query="select CENSO_ACT,VOTOS_ACT,PVOTOS_ACT,PVBLAN_ACT,PVNULOS_ACT, "+\
    "AUTONOMIA,PROVINCIA,MUNICIPIO,DISTRITO "+\
    "from escrutinio as e "+\
    "where"+\
    " e.CENSO_ACT>0;"
    return pd.read_sql_query(query,conn)

def get_taxonomy(sitio):
    """
    get AUTONOMIA,PROVINCIA,MUNICIPIO
    """
    query="select * from municipios where nombre='"+sitio+"'"
    df = pd.read_sql_query(query,conn)
    lugar=df.to_dict('index')
    
    #print("lugar",lugar)
    #Check if it corresponds in municipios to several items, e.g. tramos
    if (len(lugar.keys())>1):
        print(lugar)
        raise RuntimeError('Más de un lugar con ese nombre: '+sitio)
    else:
        lugar=lugar[0]
    return lugar

def get_distritos(lugar,verbose=False):
    #Buscar todos los distritos asociados en resultados con esa combinacion de AUTONOMIA,PROVINCIA,MUNICIPIO
    query="select distinct distrito from resultados where "+\
      "AUTONOMIA="+lugar['AUTONOMIA']+\
      " and PROVINCIA="+lugar['PROVINCIA']+\
      " and MUNICIPIO="+lugar['MUNICIPIO']
    distritos = pd.read_sql_query(query,conn)
    if verbose:
        print("distritos",distritos)
    if 'distrito' in distritos.columns:
        return distritos['distrito'].values
    else:
        return ["00"]

def get_ganador_lugar(votos_lugar,verbose=False):
    """Dados los resultados de un lugar, 
    devuelve las siglas del ganador"""
    if verbose: print(votos_lugar)
    ganador=votos_lugar.loc[votos_lugar['PVOTOS_ACT'].idxmax()]['sigla']
    return ganador


def get_escrutinio_lugar(lugar,distrito='00'):
    query="select MESAS_ACT,CENSO_ACT,VOTOS_ACT,PVOTOS_ACT,VBLAN_ACT,PVBLAN_ACT,VNULOS_ACT,PVNULOS_ACT "+\
    "from escrutinio as e "+\
    "where"+\
    " e.AUTONOMIA="+lugar['AUTONOMIA']+\
    " and e.PROVINCIA="+lugar['PROVINCIA']+\
    " and e.DISTRITO="+distrito+\
    " and e.MUNICIPIO="+lugar['MUNICIPIO']+\
    ";"
    escrutinio_lugar = pd.read_sql_query(query,conn)
    return escrutinio_lugar

def get_name(lugar,verbose=False):
    """
    Get the name of a lugar={'AUTONOMIA':00,'PROVINCIA':00,'MUNICIPIO'}
    """
    query="select distinct nombre from municipios where "+\
      "AUTONOMIA="+lugar['AUTONOMIA']+\
      " and PROVINCIA="+lugar['PROVINCIA']+\
      " and MUNICIPIO="+lugar['MUNICIPIO']

    sitio = pd.read_sql_query(query,conn)
    if verbose:
        print("sitio: ",sitio)
    if len(sitio)==0:
        sitio="@A:"+lugar['AUTONOMIA']+"-P:"+lugar['PROVINCIA']+"-M:"+lugar['MUNICIPIO']
    else:
        sitio = sitio['nombre'].values[0]
    return sitio

def single_plot(k,nbins,title,p_datos_nacional,p_datos_lugar,datos_lugar):
    t_range = np.linspace(0,100,nbins)
    print(title,end=' ',flush=True)
    n, bins, patches = plt.hist(p_datos_nacional, nbins, density=True, facecolor='b', alpha=0.75)
    plt.axvline(x=p_datos_lugar,linewidth=5,color='r')
    #Probability Density Function, with a Gaussian Kernel
    bw_values =  [None]#,.1,'scott','silverman'] #kernels for the PDF
    for i, bw in enumerate(bw_values):
        kde = stats.gaussian_kde(p_datos_nacional,bw_method=bw) 
        plt.plot(t_range,kde(t_range),lw=2, label='bw = '+str(bw))
        # print(np.sum(kde(t_range))) #this value is not 1 because the kernel (Gaussian) extends to infinity
        # enven when a normalized histogram, whose area sums up =1, the kernel makes it >1 (or less when kernel(x) extends <0)
    minx=min([np.percentile(p_datos_nacional,1),p_datos_lugar])
    maxx=max([np.percentile(p_datos_nacional,99),p_datos_lugar])
    plt.xlim(minx, maxx)
    plt.title(title+" "+str(datos_lugar))
    plt.grid(True)

def distrito_spread(sitio,todos,distrito='00',show_plot=False, verbose=False):
    cols=4   #plot columns
    rows=3 #len(votos_lugar)/4 +1
    nbins=100 #bins for the histograms
    folder='plots/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    fig=plt.figure(figsize=(15,10))
    print(sitio+" "+str(distrito))
    fig.subplots_adjust(top=1.2)
    #get indexer
    lugar=get_taxonomy(sitio)
    loc=lugar['AUTONOMIA']+"_"+\
        lugar['PROVINCIA']+"_"+\
        lugar['MUNICIPIO']+"_"+\
        distrito
    if verbose:
                print(loc)
    p_todos_no_votos=todos['PVOTOS_ACT']
    p_este_no_votos=todos.loc[loc,'PVOTOS_ACT']
    este_no_votos=todos.loc[loc,'VOTOS_ACT']
    k=1
    title=sitio+" "+str(distrito)+'-> Abs.'
    plt.subplot(rows,cols,k)
    single_plot(k,nbins,title,p_todos_no_votos,p_este_no_votos,este_no_votos)
    #each party
    #the complex call gets the first cols*rows names of the parties with most % of votes
    for party in todos.loc[loc].filter(like='v_').sort_values(ascending=False).index[0:(cols*rows)-1]:
        pparty='p_'+party[2:]
        vparty=party
        if verbose:
                print("k, p: ",k,todos.loc[loc,pparty])
        partido=party[2:]
        p_todos_partido=todos[todos[pparty]>0][pparty]
        p_este_partido=todos.loc[loc,pparty]
        este_partido=todos.loc[loc,vparty]
        k=k+1
        plt.subplot(rows,cols,k)
        single_plot(k,nbins,partido,p_todos_partido,p_este_partido,este_partido)
    
    plt.savefig(folder+str(sitio)+"-"+str(distrito)+'.png',bbox_inches = 'tight')
    if show_plot:
        plt.show()
    plt.gcf().clear()
    


In [ ]:
todos_votos     = get_todos_votos()
todos_votos

In [ ]:
no_votantes = get_no_votantes()
no_votantes

In [ ]:
#not used make_loc_key
def make_loc_key(dataframe,key='loc'):
    dataframe[key]=dataframe['AUTONOMIA']+"_"+\
                   dataframe['PROVINCIA']+"_"+\
                   dataframe['MUNICIPIO']+"_"+\
                   dataframe['DISTRITO']
    return dataframe

#todos=todos_votos.merge(no_votantes,on=['AUTONOMIA','PROVINCIA','MUNICIPIO','DISTRITO'],how='outer',suffixes=['_v','_nv'])
no_votantes=make_loc_key(no_votantes)
todos_votos=make_loc_key(todos_votos)
todos_votos


In [ ]:
#Use 'loc' as the index
todos=no_votantes
todos.index=todos['loc']
todos_votos.index=todos_votos['loc']

In [ ]:
#order of parties, by total votes
partidos=todos_votos[['VOTOS_ACT','sigla']].groupby(['sigla']).sum()\
    .sort_values(by='VOTOS_ACT',ascending=False).index

In [ ]:
for sigla in partidos:
    todos["p_"+sigla]=0
    todos["v_"+sigla]=0
todos.head(2)

In [ ]:
i=0
t=len(todos_votos)
for index,row in todos_votos.iterrows():
    todos.loc[index,"p_"+row['sigla']]=row['PVOTOS_ACT']
    todos.loc[index,"v_"+row['sigla']]=row['VOTOS_ACT']
    i+=1
    if i%1000==0: print("Filling votes: {:.1f}% ({} of {}) done".format(i/t*100,i,t),end="\r")
print("done") 
todos


In [ ]:
todos.hist(column=['PVOTOS_ACT','PVBLAN_ACT','PVNULOS_ACT'],figsize=(10,10),bins=100,density=1);

---

# Ejemplo con un sitio

In [ ]:
sitio='Madrid'

In [ ]:
lugar=get_taxonomy(sitio)
lugar

In [ ]:
get_distritos(lugar)

In [ ]:
votos_lugar=get_votos_lugar(lugar,todos,distrito='01')
votos_lugar

In [ ]:
distrito_spread(sitio,todos,distrito='00',show_plot=True ,verbose=True)

In [ ]:
#distrito_spread(sitio,distrito,get_votos_lugar(lugar),todos_votos,no_votantes,get_escrutinio_lugar(lugar))
init()
#nombres por volumen de votos
nombres=pd.read_sql_query("select distinct nombre from municipios ORDER BY RAND();",conn)[1:10]
for index,row in nombres.iterrows():
    sitio=row['nombre']
    lugar=get_taxonomy(sitio)
    distritos = get_distritos(lugar)
    for distrito in distritos:  
        distrito_spread(sitio,todos,distrito=distrito,show_plot=False,verbose=False)
        

In [ ]:
sitio='Valencia'
lugar=get_taxonomy(sitio)
print(lugar)
distritos = get_distritos(lugar)
#print("{} distritos, usando: {}".format(len(distritos),distritos[-1]))
#distrito=distritos[-1] 
print("Numero de distritos: ",len(distritos))
for distrito in distritos: 
        distrito_spread(sitio,todos,distrito=distrito,show_plot=True)

# Distritos estratégicos

Buscar los distritos más estratégicos para mandar un mensaje democrático que ayude al partido en cuestión.

In [ ]:
pvotos=todos['PVOTOS_ACT'].values
def get_maxp(pvotos,show_plot=False,party=""):
    print(".",end="")
    if len(pvotos)==1:
        return pvotos[0]
    kde = stats.gaussian_kde(pvotos,bw_method=None) 
    nbins=1000
    t_range = np.linspace(0,100,nbins)
    pdf = kde(t_range)
    max_pdf = max(pdf)
    max_p=list(pdf).index(max_pdf)*100/nbins
    if show_plot:
        if party != "":
            plt.title(party)
        plt.plot(t_range,kde(t_range));
        plt.axvline(x=max_p,linewidth=5,color='r');
        plt.show();
    return max_p

maxp_novotos=get_maxp(pvotos,show_plot=True,party='no votantes')
print(maxp_novotos)
no_votantes["delta_pvotos"]=maxp_novotos-no_votantes['PVOTOS_ACT']
no_votantes["delta_votos"]=no_votantes['CENSO_ACT']*no_votantes["delta_pvotos"]/100
no_votantes.sort_values('delta_votos',ascending=False).head(10)

In [ ]:
# Histograma nacional de votos para un partido
partidos_siglas = todos_votos['sigla'].unique()
partidos = pd.DataFrame({'sigla':partidos_siglas})
partidos['maxp']=0
    
def get_party_votes(party,all_votes):
    return all_votes[all_votes['sigla']==party]['PVOTOS_ACT'].values
    
party="PODEMOS"
print(get_party_votes(party,todos_votos))
get_maxp(get_party_votes(party,todos_votos),party=party,show_plot=True)

In [ ]:
#Maxima probabilidad de voto por partido
partidos['maxp'] = partidos.apply(lambda x: get_maxp(get_party_votes(x['sigla'],todos_votos),\
                                                               party=x['sigla'],show_plot=False), axis=1)
partidos=partidos.sort_values('maxp',ascending=False)\
                    .reset_index(drop=True)
partidos

**Hipótesis**: Los no votantes, si votan, votan proporcionalmente a lo votado **en su distrito,** y no proporcionalmente a lo que vota España.

# Clase A: Distritos de ganancia absoluta

Aquellos donde se vota poco, pero gana nuestro partido

In [ ]:
"""
1. Fijar partido
2. Filtrar donde novotos < maxp
3. Clase A: Filtrar donde partido es ganador
4. Buscar el delta de ganancia máxima: Si el distritio salta a maxp, el % que va a los ganadores..
4a. Ordenar por los votos ganados.
"""
#no_votantes = get_no_votantes()

#1 Fijar partido
partido="PSOE"


#2 Filtrar donde novotos < maxp
cols=['AUTONOMIA','PROVINCIA','MUNICIPIO','DISTRITO','PVOTOS_ACT','VOTOS_ACT','CENSO_ACT']
potenciales_no = no_votantes[no_votantes['PVOTOS_ACT']<maxp_novotos][cols]
potenciales_no.reset_index(drop=True,inplace=True)
print("{} ({:.0f}%) lugares por debajo de la media de {}% de participación".format(len(potenciales_no),
                                                                            len(potenciales_no)/len(no_votantes)*100,
                                                                            maxp_novotos))
#no usar agregados
potenciales_no=potenciales_no[potenciales_no["MUNICIPIO"]!='000']

#3 Filtrar donde gana partido
potenciales_partido=potenciales_no.head(0)
for index,row in potenciales_no.iterrows():
    lugar={'AUTONOMIA': row['AUTONOMIA'],'PROVINCIA': row['PROVINCIA'],'MUNICIPIO': row['MUNICIPIO']}
    votos_lugar=get_votos_lugar(lugar,todos_votos,distrito=row['DISTRITO'],verbose=False)
    row['ganador']=partido
    if len(votos_lugar)!=0:
        ganador_lugar=get_ganador_lugar(votos_lugar,verbose=False)
        if ganador_lugar == partido:
            potenciales_partido=potenciales_partido.append(row)
    print("{} of {}, ({} potenciales)".format(index,len(potenciales_no),len(potenciales_partido)),end='\r')



potenciales_partido.reset_index(drop=True,inplace=True)
potenciales_partido


In [ ]:
print("HAY {} distritos clase A".format(len(potenciales_partido)))
potenciales_partido.reset_index(drop=True,inplace=True)

In [ ]:
#4 Repartir el delta de nuevos votos

def get_votos_vp_partido(votos_lugar,partido):
    """
    returns [votos,pvotos] from a pd.votos_lugar and a "partido"
    """
    votos=votos_lugar[votos_lugar['sigla']==partido]
    if len(votos)==0:
        return [0,0]
    else:
        return [votos['VOTOS_ACT'].values[0],votos['PVOTOS_ACT'].values[0]]
    
#quitar donde ya hay más votos que el pmax en otros partidos
verbose=True
for index,row in potenciales_partido.head(10).iterrows():
    lugar={'AUTONOMIA': row['AUTONOMIA'],
         'PROVINCIA': row['PROVINCIA'],
         'MUNICIPIO': row['MUNICIPIO']}
    lugar['NOMBRE']=get_name(lugar)
    if verbose: print("lugar: {}".format(lugar))
    
    #quitar agregados, ya no deber'ia haber
    if "@" in lugar['NOMBRE']:
        print(lugar['NOMBRE'],"Drop agregado")
        potenciales.drop([index],inplace=True)
        continue
    votos_lugar=get_votos_lugar(lugar,todos_votos,distrito=row['DISTRITO'])
    #Repartir votos nuevos
    deltap=maxp_novotos - row['PVOTOS_ACT'] 
    delta=round(deltap)*row['CENSO_ACT'] 
    partidos['delta']=0
    print("{}, con {} censo, gana {:.2f}% ({}) votos:".format(lugar['NOMBRE'],row['CENSO_ACT'],deltap,delta))
    for index2,row2 in votos_lugar.iterrows():
        partido2=row2['sigla']
        ganancia=round(delta*row2['PVOTOS_ACT']/100)
        if verbose:
            print("\t{}, con {}% gana {} votos ".format(row2['sigla'],row2['PVOTOS_ACT'],ganancia))
        potenciales["d_"+partido2]=ganancia
        partidos[partidos['sigla']==row2['sigla']]['delta']+=ganancia
        
partidos

# Votos

Imagina que necesitamos x=1k votos.

Lo que hago es que voy reyenando votos 1 a 1,  empezando por los distritos
  1. donde es pre max_p en el partido y los no votantes. (VOTEN MAS)
  2. donde es pre max_p en el partido y el c_p de no votantes es el mejor. (VOTEN MAS)
  3. donde es pre max_p y d_P es menor que en el resto de partidos. (VOTEN A MI)
  4. donde es su d_P es menor que en el resto de partidos. (VOTEN A MI)
   



In [ ]:
# Lugar con % de participación menor que la media, que no sean agregados, y más votos
query="select AUTONOMIA,PROVINCIA,MUNICIPIO,DISTRITO,pvotos_act,votos_act \
     from escrutinio \
     where MUNICIPIO<>'000'\
     order by PVOTOS_ACT asc, VOTOS_ACT desc limit 1"
lugares = pd.read_sql_query(query,conn).to_dict('records')
for lugar in lugares:
    print(lugar)
    sitio=get_name(lugar)
    distritos = get_distritos(lugar)
    print("Numero de distritos: ",len(distritos))
    for distrito in distritos:
        escrutinio_lugar=get_escrutinio_lugar(lugar,distrito=distrito)
        votos_lugar     =get_votos_lugar(lugar,todos_votos,distrito=distrito)  
        distrito_spread(sitio,votos_lugar,todos_votos,no_votantes,escrutinio_lugar,distrito=distrito,show_plot=True)

In [ ]:
round

In [ ]:
votos_lugar

In [ ]:
#merge non_voters ike a political party
todos_votos     = get_todos_votos()
print(no_votantes.columns)
print(todos_votos.columns)

pd.merge(no_votantes, todos_votos, on=['AUTONOMIA', 'PROVINCIA', 'MUNICIPIO','DISTRITO'])

---
# Ley de d´HONT

Según [la legislación de la JEC](http://www.juntaelectoralcentral.es/cs/jec/loreg/contenido?packedargs=idContenido=546541&letra=S), el sistema es similar a la Ley de D´Hont.

Básicamente se dividen incrementalmente los votos por los escaños, y se elije el que tiene mayor número en cada división. 

Nota: Las elecciones europeas hasta 2024 no tienen umbral de votos (3% del censo normalmente). 

In [ ]:
def dhont(nSeats,votes,verbose=False):
    """
    nSeats is the number of seats
    votes is a dictionary with the key:value {'party':votes}
    verbose is an option to print designation info
    """
    t_votes=votes.copy()
    seats={}
    for key in votes: seats[key]=0
    while sum(seats.values()) < nSeats:
        max_v= max(t_votes.values())
        next_seat=list(t_votes.keys())[list(t_votes.values()).index(max_v)]
        if next_seat in seats:
            seats[next_seat]+=1
        else:
            seats[next_seat]=1
        
        if verbose:
            print("{} Escaño: {}".format(sum(seats.values()),next_seat))
            for key in t_votes: 
                print("\t{:4.4} [{}]:\t {:.1f}".format(key,seats[key],t_votes[key]))
            print("\b")
        t_votes[next_seat]=votes[next_seat]/(seats[next_seat]+1)
    for key in votes: 
        if seats[key]==0:
            del seats[key]
    return seats

In [ ]:
# How many parties and how many seats are there?
nSeats = 8
#del votes
votes = {'a':168,'b':104,'c':72,'d':64,'e':40}
seats=dhont(nSeats,votes,verbose=False)
seats

In [ ]:
query="""
select c.sigla as partido,r.VOTOS_ACT as votos 
from resultados as r
join candidaturas as c on r.candidatura=c.candidatura
where AUTONOMIA=00 
     and PROVINCIA=00 
     and municipio=000 
     and distrito=00 
     and tramo=0 
     order by votos;"""

resultados_esp = pd.read_sql_query(query,conn)
resultados_esp_dict = \
    dict(zip(resultados_esp.partido, resultados_esp.votos))
#resultados_esp_dict=sorted(resultados_esp_dict.items(), key=lambda x: x[1], reverse=True) 


In [ ]:
#D'Hont de las ultimas elecciones
nSeats = 54
seats=dhont(nSeats,resultados_esp_dict,verbose=False)
sorted(seats.items(), key=lambda x: x[1], reverse=True) 


## GEOLOCALIZACIÓN

Los datos geograficos salen de
http://centrodedescargas.cnig.es/CentroDescargas/index.jsp

También en formato mdb.
mdb-schema --namespace geo Nomenclator_Municipios_EntidadesDePoblacion.mdb mysql > schema.sql
mkdir sql
for i in $( mdb-tables Nomenclator_Municipios_EntidadesDePoblacion.mdb ); do echo $i ; mdb-export -D "%Y-%m-%d %H:%M:%S" -H -I mysql -N geo Nomenclator_Municipios_EntidadesDePoblacion.mdb $i > sql/$i.sql; done
mysql -uroot -ppassword europeas < schema.sql
for i in $( ls sql/ ); do echo $i ; mysql -uroot -ppassword europeas < sql/$i ; done

Hay dos entidades de lugar que son siempre 0:

```sql
select distinct COMARCA,TRAMO from resultados;
+---------+-------+
| COMARCA | TRAMO |
+---------+-------+
| 00      | 0     |
+---------+-------+
1 row in set (0.51 sec)
``


In [ ]:
conn.close()